In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [ ]:
%%capture
!pip install datasets
!pip install sentence_transformers

In [ ]:
from datasets import load_dataset

dataset = load_dataset("stanfordnlp/imdb")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating unsupervised split:   0%|          | 0/50000 [00:00<?, ? examples/s]

In [ ]:
df_train = dataset['train'].to_pandas()
df_test = dataset['test'].to_pandas()

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
df_train, df_val = train_test_split(df_train, train_size=0.1, random_state=0)

In [ ]:
df_train.head(3)

,text,label
11118,Once again I have seen a movie made by people ...,0
23184,I caught this on the dish last night. I liked ...,1
7399,"A bondage, humiliation, S&M show, and not much...",0


In [ ]:
df_val.head(3)

,text,label
14149,Fräulein Doktor is as good a demonstration as ...,1
8946,I watched this knowing almost nothing about it...,0
22378,I must give How She Move a near-perfect rating...,1


In [ ]:
from sentence_transformers import SentenceTransformer
model = SentenceTransformer('paraphrase-MiniLM-L6-v2' , device="cuda")

modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/3.73k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


...

tokenizer_config.json:   0%|          | 0.00/314 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
l_texts = list(df_train['text'].values) + list(df_test['text'].values) + list(df_val['text'].values)

In [ ]:
# Sentences are encoded by calling model.encode()
embedding = []
map_text_embedding = {}

In [ ]:
from tqdm.notebook import tqdm
batch_size = 64
for i in tqdm(range(0, len(l_texts), batch_size), total=len(l_texts)//batch_size + 1):
  cur_l_text = l_texts[i:i+batch_size]
  cur_embeddings = model.encode(l_texts[i:i+batch_size])
  embedding += list(cur_embeddings)
  for (cur_text, cur_emb) in zip(cur_l_text, cur_embeddings):
    map_text_embedding[cur_text] = cur_emb

  0%|          | 0/782 [00:00<?, ?it/s]

In [ ]:
df_train['embedding'] = df_train['text'].apply(lambda x: map_text_embedding[x])
df_val['embedding'] = df_val['text'].apply(lambda x: map_text_embedding[x])
df_test['embedding'] = df_test['text'].apply(lambda x: map_text_embedding[x])

In [ ]:
node_path = '/content/drive/MyDrive/node_classification/text_classification'
df_train.to_pickle(f"{node_path}/data/imdb_embs_train.pkl")
df_val.to_pickle(f"{node_path}/data/imdb_embs_val.pkl")
df_test.to_pickle(f"{node_path}/data/imdb_embs_test.pkl")